<a href="https://colab.research.google.com/github/MukhinaMaria/Lang./blob/main/%D0%A0%D0%B5%D1%88%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D1%87_%D0%9E%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B8_%D0%B5%D1%81%D1%82%D0%B5%D1%81%D1%82%D0%B2%D0%B5%D0%BD%D0%BD%D0%BE%D0%B3%D0%BE_%D1%8F%D0%B7%D1%8B%D0%BA%D0%B0_%D0%9C%D1%83%D1%85%D0%B8%D0%BD%D0%B0_%D0%9C_%D0%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# https://drive.google.com/file/d/1kD5QBEvBQCGaAzsYskIBCDo9sOeCwqKP/view?usp=sharing
# https://drive.google.com/file/d/1N_eGBLKUuGzlL5dLzQvC4UfcZdZXf6eS/view?usp=sharing

In [ ]:
!pip -q install pymorphy2
!pip -q install sentencepiece
!pip -q install transformers
!pip -q install natasha

!gdown 1kD5QBEvBQCGaAzsYskIBCDo9sOeCwqKP
!gdown 1N_eGBLKUuGzlL5dLzQvC4UfcZdZXf6eS

Downloading...
From: https://drive.google.com/uc?id=1kD5QBEvBQCGaAzsYskIBCDo9sOeCwqKP
To: /content/bert_dataset.py
100% 891/891 [00:00<00:00, 2.74MB/s]
Downloading...
From: https://drive.google.com/uc?id=1N_eGBLKUuGzlL5dLzQvC4UfcZdZXf6eS
To: /content/bert_classifier.py
100% 5.28k/5.28k [00:00<00:00, 13.0MB/s]


In [ ]:
# основные по работе с данными (таблицы, математика)
import pandas as pd
import numpy as np

# обработка языка
import nltk # язык nlp
from nltk.corpus import stopwords
import string # строки
import re # регулярные выражения
import codecs # определяет базовые классы для стандартных кодировок Python
import pymorphy2 # русский язык
import natasha # русский язык, NER анализ TODO добавить лемматизацию natasha
# графики
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,5) # размер графиков
plt.style.use('fivethirtyeight') # стиль графиков
# модели
import sklearn

In [ ]:
# стоп-слова
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# русские стоп-слова
stopWords = set(stopwords.words('russian'))

In [ ]:
# удаление пунктуации
SYMBOLS = " ".join(string.punctuation).split(" ") + ["-", "...", "”", "”"]

In [ ]:
# лемматизатор pymorphy2, natasha
# Docs - https://pymorphy2.readthedocs.io/en/stable/
morph = pymorphy2.MorphAnalyzer() # основной класс

In [ ]:
SYMBOLS

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~',
 '-',
 '...',
 '”',
 '”']

In [ ]:
def str_preprocess(text: str)->list:
    """
    Очистка текста
    """
    reg = re.compile('[^а-яА-яa-zA-Z0-9 ]') # буквы русского, английского, числа
    text = text.lower().replace("ё", "е") # ё замена на е - с осторожностью, чат
    text = text.replace("ъ", "ь") # замена ь  - с осторожностью, чат
    text = text.replace("й", "и") # замена й  - с осторожностью, чат
    text = text.replace("\\n", "") # замена переноса строки
    text = text.replace("\\r", "") # замена спецсимоволов
    text = text.replace("\\t", "") # замена длинных табов
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'сайт', text) # url
    text = re.sub('@[^\s]+', 'пользователь', text) # пользователь s - строка
    text = reg.sub(' ', text) # выходной текста

    # Лемматизация
    text =  " ".join([word for word in text.split() if word not in SYMBOLS]) # фильтр символов
    text =[morph.parse(word)[0].normal_form for word in text.split() if word not in stopWords] # фильтр стоп-слов
    return ' '.join(text)

In [ ]:
from tqdm import tqdm # процент выполнения, отображается в процессе работы
tqdm.pandas()

In [ ]:
# Файл с данными 'текст песни3.csv' /content/drive/MyDrive/Test data/текст песни3.csv

In [ ]:
# Файл с данными 'текст песни3.csv'
df = pd.read_csv('/content/drive/MyDrive/Test data/текст песни3.csv')

In [ ]:
df

,text,label
0,"В городе оттепель, чавкает в лужах",0
1,"Серый, мартовский снег.",0
2,Город ранней весною контуженный.,0
3,Вскрылся венами рек.,0
4,"Всё перемолото, скомкано, сорвано.",0
5,Слишком долгой зимой.,0
6,"Но у меня есть ты, значит господь со мной.",0
7,Оттепель смазала контуры мира.,1
8,Словно кистью Дали.,0
9,Звуки текущие прямо с клавира.,0


In [ ]:
# данные песни, 2 темы (строка начинается с согласной и строка начинается с гласной)
human_name = {0:'Vowel', 1: 'Consonant'}
df['label_name'] = df['label'].apply(lambda x: human_name[x])

In [ ]:
# применяем очистку текста
df['clean_text'] = df.progress_apply(lambda x : str_preprocess(x.text), axis=1)

100%|██████████| 45/45 [00:00<00:00, 972.43it/s]


In [ ]:
df.head()

,text,label,clean_text,label_name
0,"В городе оттепель, чавкает в лужах",0,город оттепель чавкать лужа,Vowel
1,"Серый, мартовский снег.",0,серый мартовския снег,Vowel
2,Город ранней весною контуженный.,0,город раннея весною контуженный,Vowel
3,Вскрылся венами рек.,0,вскрыться вено река,Vowel
4,"Всё перемолото, скомкано, сорвано.",0,перемолоть скомкать сорвать,Vowel


In [ ]:
df_index = 35

df.iloc[df_index].text

'Я у любви буду прощённый.'

In [ ]:
df.to_csv('dataset_clean.csv')

In [ ]:
from sklearn.model_selection import train_test_split # делим на тестовую и тренировочную
from sklearn.feature_extraction.text import CountVectorizer # мешок слов


from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report # метрика точности, f1 мера, матрица ошибок и отчёт по классификации
from sklearn import model_selection # выбор модели и оценка параметров (можно загрузить несколько моделей)
from sklearn.linear_model import LogisticRegression # логистическая регрессия
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier # бустинг, случайный лес
from sklearn.svm import SVC # метод SVM, загружаем с ядром rbf https://scikit-learn.ru/1-4-support-vector-machines/
from sklearn.neighbors import KNeighborsClassifier # к ближайших соседей

In [ ]:
df = pd.read_csv('./dataset_clean.csv', index_col = 0)

In [ ]:
# Оставим только данные для обучения
df['text'] = df['clean_text']
df=df[['text', 'label']]

In [ ]:
df

,text,label
0,город оттепель чавкать лужа,0
1,серый мартовския снег,0
2,город раннея весною контуженный,0
3,вскрыться вено река,0
4,перемолоть скомкать сорвать,0
5,слишком долгой зимои,0
6,значит господь мной,0
7,оттепель смазать контур мир,1
8,словно кисть дать,0
9,звук текущий прямо клавир,0


In [ ]:
import string # обработка строк (ещё раз на случай если запуск кода с этой ячейки)
import spacy # обработка языка
from spacy.lang.ru.stop_words import STOP_WORDS # стоп-слова
from spacy.lang.ru import Russian # язык русский

In [ ]:
!python -m spacy download ru_core_news_lg

2023-11-19 13:06:42.249976: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-19 13:06:42.250063: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-19 13:06:42.250103: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-19 13:06:43.793556: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 817.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 48.0 MB/s eta 0:0

In [ ]:
# Создать список знаков препинания
punctuations = string.punctuation

# Загрузить список стоп-слов
nlp = spacy.load('ru_core_news_lg')

In [ ]:
# Загрузить русский токенизатор, теггер, синтаксический анализатор, NER и векторы слов
parser = Russian()

In [ ]:
STOP_WORDS

{'а',
 'авось',
 'ага',
 'агу',
 'аж',
 'ай',
 'али',
 'алло',
 'ау',
 'ах',
 'ая',
 'б',
 'бац',
 'без',
 'безусловно',
 'бишь',
 'благо',
 'благодаря',
 'ближайшие',
 'близко',
 'более',
 'больше',
 'будем',
 'будет',
 'будете',
 'будешь',
 'будто',
 'буду',
 'будут',
 'будучи',
 'будь',
 'будьте',
 'бы',
 'бывает',
 'бывала',
 'бывали',
 'бываю',
 'бывают',
 'был',
 'была',
 'были',
 'было',
 'бытует',
 'быть',
 'в',
 'вам',
 'вами',
 'вас',
 'ваш',
 'ваша',
 'ваше',
 'ваши',
 'вдали',
 'вдобавок',
 'вдруг',
 'ведь',
 'везде',
 'вернее',
 'весь',
 'взаимно',
 'взаправду',
 'видно',
 'вишь',
 'включая',
 'вместо',
 'внакладе',
 'вначале',
 'вне',
 'вниз',
 'внизу',
 'вновь',
 'во',
 'вовсе',
 'возможно',
 'воистину',
 'вокруг',
 'вон',
 'вообще',
 'вопреки',
 'вот',
 'вперекор',
 'вплоть',
 'вполне',
 'вправду',
 'вправе',
 'впрочем',
 'впрямь',
 'вресноту',
 'вроде',
 'вряд',
 'все',
 'всегда',
 'всего',
 'всей',
 'всем',
 'всеми',
 'всему',
 'всех',
 'всею',
 'всея',
 'всю',
 'всюд

In [ ]:
punctuations

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def spacy_tokenizer(sentence):
    # Создаем функцию токенизатора
    # Создаем объект-токен, который используется для создания документов с лингвистическими аннотациями.
    mytokens = parser(sentence)
    print(sentence)

    result = []
    # Лемматизация каждого токена и преобразование каждого токена в нижний регистр
    for word in mytokens:
      word_text = word.orth_
      if nlp.vocab[word_text].is_stop != True and word_text not in punctuations:
        # слова, которые не обрабатывает spacy обрабатываем pymorphy2
        normal_form = morph.parse(word_text)[0].normal_form.lower().strip() if word.lemma_ != "-PRON-" else word.lower_
        print(normal_form)
        result.append(normal_form)
    return " ".join(result)

In [ ]:
from bert_dataset import CustomDataset
from bert_classifier import BertClassifier

In [ ]:
EPOCH_COUNT_BERT = 4
CLASS_LABELS = 2

# https://huggingface.co/cointegrated/rubert-tiny ruBERT
classifier = BertClassifier(
        model_path='cointegrated/rubert-tiny',
        tokenizer_path='cointegrated/rubert-tiny',
        n_classes=CLASS_LABELS,
        epochs=EPOCH_COUNT_BERT,
        model_save_path='/content/bert.pt'
)

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/468k [00:00<?, ?B/s]